#Traffic flow optimization without constraint term on quantum computing using quantum alternating operator ansatz

##Quantum Alternating Operator Ansatz
Here think about how to rise up the accuracy of optimization problems on quantum computing.

One of this answer is to use Quantum Alternating Operator Ansatz for QAOA. It can reduce the number of constraint term in the qubo formulation.

By adopting XX+YY gate, the time evolution can operate a swap of 01 and 10. This achieve a good operation on combinatorial optimization problem if we know that 00 and 11 not appear in the result from first.

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F88256%2F0b4a9d82-8ebf-8191-23d7-dbb76396e9d1.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=616a52dfc97ba36e01aba646f9affdac">

reference: https://qiita.com/gyu-don/items/c51a9e3d5d16a6d5baf6

Let's solve a simple problem using this technique



##Example
Here we try a simple hamiltonian of

```python
h = -3*q(0)-3*q(1)-2*q(0)*q(1)
```

The minimum answer is the combination of (1,1) and this hamiltonian takes minimum value as -8.

Now we use the entanglement on the initial state of |10> and |01>. And use XY mixer as a driver hamiltonian of time evolution.

In [0]:
!pip install blueqat

     |████████████████████████████████| 51kB 1.7MB/s 


In [1]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe

def an(index):
    return 0.5 * X[index] + 0.5j * Y[index]

def cr(index):
    return 0.5 * X[index] - 0.5j * Y[index]

op = (cr(1) * an(0) + cr(0) * an(1)).to_expr().simplify()

class QaoaQaoaAnsatz(AnsatzBase):
    def __init__(self, hamiltonian, step=1, init_circuit=None):
        super().__init__(hamiltonian, step * 2)
        self.hamiltonian = hamiltonian.to_expr().simplify()
        if not self.check_hamiltonian():
            raise ValueError("Hamiltonian terms are not commutable")

        self.step = step
        self.n_qubits = self.hamiltonian.max_n() + 1
        if init_circuit:
            self.init_circuit = init_circuit
            if init_circuit.n_qubits > self.n_qubits:
                self.n_qubits = init_circuit.n_qubits
        else:
            self.init_circuit = Circuit(self.n_qubits).h[:]
        self.init_circuit.make_cache()
        self.time_evolutions = [term.get_time_evolution() for term in self.hamiltonian]
        self.time_evolutions2 = [term.get_time_evolution() for term in op]
    def check_hamiltonian(self):
        """Check hamiltonian is commutable. This condition is required for QaoaAnsatz"""
        return self.hamiltonian.is_all_terms_commutable()

    def get_circuit(self, params):
        c = self.init_circuit.copy()
        betas = params[:self.step]
        gammas = params[self.step:]
        for beta, gamma in zip(betas, gammas):
            beta *= np.pi
            gamma *= 2 * np.pi
            for evo in self.time_evolutions:
                evo(c, gamma)
            for evo2 in self.time_evolutions2:
                evo2(c, beta)
        return c

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
h = h.to_expr().simplify()
runner = Vqe(QaoaQaoaAnsatz(h,4,Circuit().h[0].cx[0,1].x[0]))
result = runner.run()

# get probability
print(result.most_common(12))

print('Result by QAOA')
print(runner.ansatz.get_energy_sparse(result.circuit))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

(((1, 0), 0.49999999999999906), ((0, 1), 0.49999999999999906), ((0, 0), 3.42815530100928e-31), ((1, 1), 3.274080905458301e-33))
Result by QAOA
-2.999999999999995
Result by numpy
-8.0


Numpy shows the minimum result as -8 but this QAOA has limitation of |01> or |10> as a result using entanglement and XYmixer. |00> or |11> doesn't appear in the result.

Using this we can get much more accurate result on a specific problem with constraints.

<img width="1281" alt="スクリーンショット 2020-02-25 0.39.32.png" src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/85efc267-c34d-dbc8-1bb0-25bc22cfb1c0.png">

##Traffic flow optimization without constraint term on quantum computing using quantum alternating operator ansatz
Here we adopted traffic flow optimization problem as a case study, but this technique is a general one and can be applied to a variety of problems.

References
As a model we use traffic flow modeling from VW paper.

Traffic flow optimization using a quantum annealer  
Florian Neukart, Gabriele Compostella, Christian Seidel, David von Dollen, Sheir Yarkoni, Bob Parney  
(Submitted on 4 Aug 2017 (v1), last revised 9 Aug 2017 (this version, v2))  
https://arxiv.org/pdf/1708.01625.pdf

As an algorithm we use QAOA for universal gate model.  
A Quantum Approximate Optimization Algorithm  
Edward Farhi, Jeffrey Goldstone, Sam Gutmann  
https://arxiv.org/abs/1411.4028  

And technique to use XY mixer.  
From the Quantum Approximate Optimization Algorithm to a Quantum Alternating   Operator Ansatz
Stuart Hadfield, Zhihui Wang, Bryan O’Gorman, Eleanor G. Rieffel, Davide Venturelli, Rupak Biswas  
https://arxiv.org/abs/1709.03489  

And prepare an entangled state.  
Portfolio rebalancing experiments using the
Quantum Alternating Operator Ansatz  
Mark Hodson∗, Brendan Ruck∗, Hugh (Hui Chuan) Ong†, David Garvin∗, Stefan Dulman†
∗Rigetti Computing
†Commonwealth Bank of Australia  
https://arxiv.org/pdf/1911.05296.pdf

##Modeling and formulation
We use the model from the VW. We prepare start point A and goal point B. And named each street from s0 to s11.

<img src="https://miro.medium.com/max/730/0*jdvbeWCvUMG-UKis">

Let’s prepare two cars and set possible route setting for each vehicles.

car1  
route1-1(q0)：s0,s3,s6,s9  
route1–2 (q1)：s0,s3,s8,s11  

car2  
route2–1(q2):s0,s3,s8,s11  
route2–2(q3):s2,s7,s10,s11

q0 to q3 denote qubits. If the route selected the value of qubit is 1 else 0. If route1–1 is selected for car1 and route2–2 is selected for car2 the answer is, (1,0,0,1)

Cost=(q0+q1+q2)²+(q0+q1+q2)²+q0²+q0²+(q1+q2)²+(q1+q2+q3)²+q3²+q3²+q3²=4q0+4q1+4q2+4q3+4q0q1+4q0q2+8q1q2+2q1q3+2q2q3

And the constraint is that car1 select just 1route from q0 and q1. Car2 also select just one route from q2 and q3.

Const=(q0+q1−1)²+(q2+q3−1)²=q0−q1−q2−q3+2q0q1+2q2q3+2

Usually we connect these equation using variable K.

Cost+K∗ConstCost+K∗Const

Now we think about a new technique to remote this Const term.

##Prepare the initial state
Usually quantum annealing is searching the value almost randomly using sigmaX. And the initial state is |+> as the eigen state of hamiltonian X. As the mixer it is using time evolution of hamitonian X.

But here we think about the pattern of searching. Each car always select just one route from the possible routes. So, just one of q0 or q1 is selected as 1, also just one of q2 or q3 is selected as 1.

Here we use “Quantum Entanglement”. Entanglement add filter to the possible answer just limited to the expected value.

Here is a very simple of entanglement. By using this circuit making the entanglemnt, we get

In [2]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].m[:].run(shots=100)

Counter({'00': 46, '11': 54})

And applying X gate to the 0th qubit, we get entanglement of 01 and 10 state.

In [3]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].x[0].m[:].run(shots=100)

Counter({'01': 39, '10': 61})

By using this state preparation we can set entanglement both on q0,q1 and q2,q3 corresponding to the possible route of car1 and car2.

This time we have 4qubits and entangled state is,

In [4]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].x[0].h[2].cx[2,3].x[2].m[:].run(shots=100)

Counter({'0110': 25, '0101': 22, '1001': 22, '1010': 31})

##Swap
Instead of searching the qubit value using sigmaX we use (XX+YY)/2 gate swapping two qubits value.

By using this gate we can changing the state of 01 and 10, applying to the adiabatic process searching the global minimum on QAOA algorithm.

##Code
Here is the blueqat code, we are using types of time evolution operator both as classical hamiltonian of Z operator and mixer hamiltonian.

In [5]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe

def an(index):
    return 0.5 * X[index] + 0.5j * Y[index]

def cr(index):
    return 0.5 * X[index] - 0.5j * Y[index]

op1 = (cr(1) * an(0) + cr(0) * an(1)).to_expr().simplify()
op2 = (cr(3) * an(2) + cr(2) * an(3)).to_expr().simplify()

class QaoaQaoaAnsatz(AnsatzBase):
    def __init__(self, hamiltonian, step=1, init_circuit=None):
        super().__init__(hamiltonian, step * 2)
        self.hamiltonian = hamiltonian.to_expr().simplify()
        if not self.check_hamiltonian():
            raise ValueError("Hamiltonian terms are not commutable")

        self.step = step
        self.n_qubits = self.hamiltonian.max_n() + 1
        if init_circuit:
            self.init_circuit = init_circuit
            if init_circuit.n_qubits > self.n_qubits:
                self.n_qubits = init_circuit.n_qubits
        else:
            self.init_circuit = Circuit(self.n_qubits).h[:]
        self.init_circuit.make_cache()
        self.time_evolutions = [term.get_time_evolution() for term in self.hamiltonian]
        self.time_evolutions1 = [term.get_time_evolution() for term in op1]
        self.time_evolutions2 = [term.get_time_evolution() for term in op2]
    def check_hamiltonian(self):
        """Check hamiltonian is commutable. This condition is required for QaoaAnsatz"""
        return self.hamiltonian.is_all_terms_commutable()

    def get_circuit(self, params):
        c = self.init_circuit.copy()
        betas = params[:self.step]
        gammas = params[self.step:]
        for beta, gamma in zip(betas, gammas):
            beta *= np.pi
            gamma *= 2 * np.pi
            for evo in self.time_evolutions:
                evo(c, gamma)
            for evo1 in self.time_evolutions1:
                evo1(c, beta)
            for evo2 in self.time_evolutions2:
                evo2(c, beta)
        return c

h = 4*q(0)+4*q(1)+4*q(2)+4*q(3)+4*q(0)*q(1)+4*q(0)*q(2)+8*q(1)*q(2)+2*q(1)*q(3)+2*q(2)*q(3)
h = h.to_expr().simplify()
runner = Vqe(QaoaQaoaAnsatz(h,4,Circuit().h[0].cx[0,1].x[0].h[2].cx[2,3].x[2]))
result = runner.run()

# get probability
print(result.most_common(12))

print('Result by QAOA')
print(runner.ansatz.get_energy_sparse(result.circuit))

(((1, 0, 0, 1), 0.9725719073393897), ((0, 1, 0, 1), 0.024015993375551083), ((1, 0, 1, 0), 0.002505733361743824), ((0, 1, 1, 0), 0.0009063659233088916), ((1, 0, 0, 0), 3.5709449507923647e-31), ((1, 1, 0, 1), 1.9528948916201288e-31), ((0, 0, 0, 1), 1.0405633442510895e-31), ((1, 0, 1, 1), 4.5019364766436164e-32), ((0, 1, 1, 1), 1.9322494020172565e-32), ((0, 1, 0, 0), 1.9259299443872359e-32), ((1, 1, 1, 0), 1.898225421321941e-32), ((0, 0, 1, 0), 9.408022689142076e-33))
Result by QAOA
8.065305847584497


Here we get the best solution of (1,0,0,1) with the 98% of possibility. This is the best route removing the traffic jam.

##Discussion
By using this, the depth of the circuit will be long, but we don’t have to formulate the constraint term. And the possibility of getting accurate result dramatically increase.